In [4]:
import yfinance as yf
import numpy as np
import pandas as pd

In [22]:
def get_price_data(tickers, start_date, end_date):
    """
    Download adjusted close prices for given tickers and date range.
    Returns a DataFrame with dates as index and tickers as columns.
    """
    data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=False)['Adj Close']
    return data.dropna()



In [6]:

def compute_daily_returns(price_df):
    """
    Compute daily returns from price data.
    """
    return price_df.pct_change().dropna()


In [7]:

def portfolio_returns(daily_returns, weights):
    """
    Calculate portfolio daily returns given individual daily returns and weights.
    weights should be a list or array in the same order as columns of daily_returns.
    """
    weights = np.array(weights)
    return daily_returns.dot(weights)


In [ ]:

def portfolio_volatility(portfolio_ret):
    """
    Annualized volatility of portfolio returns.
    """

    return portfolio_ret.std() * np.sqrt(len(portfolio_ret))


In [9]:

def historical_var(portfolio_ret, confidence_level=0.95):
    """
    Historical Value at Risk (VaR).
    Returns the loss at the specified confidence level.
    """
    # Calculate the percentile of losses (negative returns)
    var = np.percentile(portfolio_ret, 100 * (1 - confidence_level))
    return var

In [10]:
tickers = ['AAPL', 'MSFT', 'TSLA']
weights = [0.4, 0.4, 0.2]
start_date = '2022-01-01'
end_date = '2023-01-01'


In [24]:

prices = get_price_data(tickers, start_date, end_date)


[*********************100%***********************]  3 of 3 completed


In [38]:
prices
#returns

Ticker,AAPL,MSFT,TSLA
Date,,,
2022-01-03,178.645630,325.038025,399.926666
2022-01-04,176.378342,319.464600,383.196655
2022-01-05,171.686676,307.200989,362.706665
2022-01-06,168.820709,304.773560,354.899994
2022-01-07,168.987549,304.928955,342.320007
...,...,...,...
2022-12-23,130.173782,233.975891,123.150002
2022-12-27,128.367172,232.241150,109.099998
2022-12-28,124.428207,229.859497,112.709999


In [25]:

returns = compute_daily_returns(prices)


In [36]:
returns.iloc[0]

Ticker
AAPL   -0.012692
MSFT   -0.017147
TSLA   -0.041833
Name: 2022-01-04 00:00:00, dtype: float64

In [37]:
-0.012692 * weights[0] + -0.017147 * weights[1] + -0.041833 * weights[2]

-0.0203022

In [33]:
port_ret

Date
2022-01-04   -0.020302
2022-01-05   -0.036689
2022-01-06   -0.014143
2022-01-07   -0.006490
2022-01-10    0.006408
                ...   
2022-12-23   -0.003723
2022-12-27   -0.031335
2022-12-28   -0.009758
2022-12-29    0.038547
2022-12-30    0.001245
Length: 250, dtype: float64

In [40]:
vol

np.float64(0.3679667893100652)

In [26]:

port_ret = portfolio_returns(returns, weights)
vol = portfolio_volatility(port_ret)
var_95 = historical_var(port_ret)


In [27]:

print(f"Annualized Volatility: {vol:.2%}")
print(f"Historical VaR (95% confidence): {var_95:.2%}")

Annualized Volatility: 36.80%
Historical VaR (95% confidence): -3.87%


In [39]:
from scipy.stats import norm

def parametric_var(portfolio_returns, confidence_level=0.95):
    mean_return = np.mean(portfolio_returns)
    std_dev = np.std(portfolio_returns)
    z_score = norm.ppf(1 - confidence_level)
    var = -(mean_return + z_score * std_dev)
    return var


In [43]:

import pandas_market_calendars as mcal

def is_trading_period(start, end):
    try:
        nyse = mcal.get_calendar('NYSE')
        schedule = nyse.schedule(start_date=start, end_date=end)
        return not schedule.empty
    except Exception as e:
        print(f"Error checking trading calendar: {e}")
        return False
